In [1]:
# [
#     "blocks.0.hook_resid_pre",
#     "blocks.0.ln1.hook_scale",
#     "blocks.0.ln1.hook_normalized",
#     "blocks.0.attn.hook_q",
#     "blocks.0.attn.hook_k",
#     "blocks.0.attn.hook_v",
#     "blocks.0.attn.hook_rot_q",
#     "blocks.0.attn.hook_rot_k",
#     "blocks.0.attn.hook_attn_scores",
#     "blocks.0.attn.hook_pattern",
#     "blocks.0.attn.hook_z",
#     "blocks.0.hook_attn_out",
#     "blocks.0.hook_resid_mid",
#     "blocks.0.ln2.hook_scale",
#     "blocks.0.ln2.hook_normalized",
#     "blocks.0.mlp.hook_pre",
#     "blocks.0.mlp.hook_pre_linear",
#     "blocks.0.mlp.hook_post",
#     "blocks.0.hook_mlp_out",
#     "blocks.0.hook_resid_post",
# ]

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import matplotlib.pyplot as plt
import torch

In [3]:
# model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"
# model_name = "microsoft/Phi-3-mini-4k-instruct"
model_name = "google/gemma-2-9b-it"
# model_name = "Qwen/Qwen2-7B-Instruct"
device = "cuda"

In [4]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map=device,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    attn_implementation="flash_attention_2",
)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, padding_side="left")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
generation_config = GenerationConfig(do_sample=False, num_beams=1)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [15]:
text = "半小时后在小区门口见，可以吗？"
text_tokenized = tokenizer.encode(text, return_tensors="pt").to(device)
output = model.generate(text_tokenized, generation_config=generation_config, max_new_tokens=5)
text_decoded = tokenizer.decode(output[0], skip_special_tokens=True)
print(text_decoded)

半小时后在小区门口见，可以吗？

好的，我在小区


In [ ]:
def decode_token_with_logitlens(model, device, tokenizer, input, tokens_to_gen=None):
    """
    outputs a dictionary with {'decoded_tokens': [num_layers, seq_len], 'decoded_logits': [num_layers, seq_len]}
    """
    inputs = tokenizer(input, return_tensors="pt").to(device)

    text = tokenizer.decode(inputs["input_ids"][0])
    # run the loop to generate new tokens after input, append to input and decode
    if tokens_to_gen is not None:
        # generate new tokens all at once; then append them to input, then logitlens them all
        output = model.generate(
            inputs["input_ids"],
            generation_config=generation_config,
            max_new_tokens=tokens_to_gen,
        )
        new_token = tokenizer.decode(output[0][-tokens_to_gen:])
        text += new_token
        inputs = tokenizer(text, return_tensors="pt").to(device)
    text_tokens = [tokenizer.decode(id) for id in inputs["input_ids"][0]]

    # apply decoder lens
    classifier_head = model.lm_head  # Linear(in_features=3072, out_features=32064, bias=False)

    hidden_states = model(**inputs, output_hidden_states=True).hidden_states
    decoded_intermediate_token = {}
    decoded_intermediate_logit = {}
    with torch.no_grad():
        for layer_id in range(len(hidden_states)):
            hidden_state = hidden_states[layer_id]
            decoded_value = classifier_head(hidden_state)  # [batch, seq_len, vocab_size]
            # get probabilities
            # decoded_values = torch.nn.functional.softmax(decoded_value, dim=-1)
            decoded_values = decoded_value
            # take max element
            argmax = torch.argmax(decoded_values, dim=-1)[0]  # select first element in batch
            # decode all tokens
            decoded_token = [tokenizer.decode(el) for el in argmax]
            decoded_logit = [
                decoded_values[0, it, argmax[it]].item() for it in range(len(argmax))
            ]  # list of layers, per layer the sequence_length
            decoded_intermediate_token[layer_id] = decoded_token
            decoded_intermediate_logit[layer_id] = decoded_logit

    tokens = list(decoded_intermediate_token.values())  # [num_layers, seq_len]
    logits = list(decoded_intermediate_logit.values())  # [num_layers, seq_len]
    return {"text_tokens": text_tokens, "decoded_tokens": tokens, "decoded_logits": logits}

In [17]:
# let's test patch decode
dict_output = decode_token_with_logitlens(model, device, tokenizer, text, tokens_to_gen=5)
decoded_tokens = dict_output["decoded_tokens"]
decoded_logits = dict_output["decoded_logits"]
text_tokens = dict_output["text_tokens"]

to_viz = (len(text_tokens) - 10, len(text_tokens))

print(len(text_tokens), text_tokens)

18 ['<bos>', '<bos>', '半', '小时', '后', '在', '小区', '门口', '见', '，', '可以', '吗', '？', '\n\n', '好的', '，', '我在', '小区']


In [ ]:
import matplotlib.font_manager as fm
plt.style.use("ggplot")

font_files = fm.findSystemFonts(fontpaths="/usr/share/fonts")

for f in sorted(font_files):
    try:
        if 'NotoSans' in f and 'UI' not in f:
            fm.fontManager.addfont(f)
            # print(f)
    except Exception:
        print(f)

In [19]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap

font_prop = fm.FontProperties(family=sorted(fm.get_font_names()))

# Extract data for visualization
tokens_viz = [tok[to_viz[0] : to_viz[1]] for tok in decoded_tokens]
logits_viz = np.array([log[to_viz[0] : to_viz[1]] for log in decoded_logits])
token_indices = list(range(to_viz[0], to_viz[1]))
token_labels = text_tokens[to_viz[0] : to_viz[1]]

# Create a custom colormap
colors = ["#4575b4", "#ffffbf", "#d73027"]  # Blue to yellow to red
n_bins = 100
cmap = LinearSegmentedColormap.from_list("custom", colors, N=n_bins)

# Set up the plot with increased width
fig, ax = plt.subplots(figsize=(len(decoded_logits[0]), len(decoded_logits) // 2), dpi=400)
sns.set_style("whitegrid")

# Create heatmap
sns.heatmap(logits_viz, cmap=cmap, ax=ax, cbar_kws={"label": "Logit Value"})

# Customize the plot
ax.set_title("Token Logits Across Model Layers", fontsize=16, pad=20)
ax.set_xlabel("Tokens", fontsize=16, labelpad=10)
ax.set_ylabel("Model Layers", fontsize=16, labelpad=10)

# Set x-axis labels at the top without rotation
ax.xaxis.tick_top()
ax.xaxis.set_label_position("top")
ax.set_xticks(np.arange(len(token_labels)) + 0.5)
ax.set_xticklabels([f"{i}: {t}" for i, t in zip(token_indices, token_labels)], ha="center", fontproperties=font_prop, fontsize=12)

# Set y-axis labels
ax.set_yticks(np.arange(len(decoded_tokens)) + 0.5)
ax.set_yticklabels(range(len(decoded_tokens)), fontproperties=font_prop, fontsize=12)


# Add text annotations
for i in range(len(decoded_tokens)):
    for j in range(len(token_labels)):
        ax.text(
            j + 0.5,
            i + 0.5,
            tokens_viz[i][j],
            ha="center",
            va="center",
            color="black",
            fontsize=10,
            fontproperties=font_prop,
        )

# Adjust layout and display
plt.tight_layout()
plt.show()